https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/ <br>
https://github.com/yyeung-lam/earnings-call-text-mining <br>
https://towardsdatascience.com/text-mining-and-classification-on-earnings-call-transcripts-e63d9856d7d8 

Install spacy, textacy and nltk. <br>
Install spacy module through command  python -m spacy download en_core_web_sm . <br>
If the popup window remain, manually close them or the programm will not advance. <br>


In [1]:
import pandas as pd
import textacy, spacy
import tkinter as tk
from tkinter import filedialog
from tkinter import *
import en_core_web_sm
from textblob import TextBlob
from textacy import TextStats
import sys, os, re

In [8]:
def browse_button():
    # Allow user to select a directory and store it in global var
    # called folder_path
    
    global folder_path
    global filename
    filename = filedialog.askdirectory()
    folder_path.set(filename)
    print(filename)
    sourcePath = folder_path.get()
    os.chdir(sourcePath)  # Provide the path here

root = Tk()

folder_path = StringVar()

lbl1 = Label(master=root,textvariable=folder_path)
lbl1.grid(row=0, column=1)

buttonBrowse = Button(text="        Set directory where the raw files are located        ", 
         
                      command = browse_button, bg = 'green',
                      fg='white', font = ('helvetica', 12, 'bold'))
buttonBrowse.grid(row=2, column=1)
mainloop()

C:/Users/Mahum/Desktop/ALL/PYTHON/NLP/default/raw


# Step One: Data Cleaning and Preprocessing


### Clean the text and save it to another folder

In [9]:
nlp = en_core_web_sm.load()
#nlp = spacy.load("en_core_web_sm")


def tokenize(text):
    paragraphs = text.split('\n')
    processed_text = ""
    for p in paragraphs:
        if len(p) < 15:
            continue
        doc = nlp(p)
        for token in doc:
            if not token.is_punct:
                replaced_tok = token.lemma_
                if token.lemma_ == '-PRON-':
                    replaced_tok = 'it'
                processed_text += (replaced_tok + " ")
            else:
                processed_text += (token.text + " ")
    processed_text = re.sub(r"(good morning( everyone)*)", "", processed_text)
    processed_text = re.sub(r"(thank you)", "", processed_text)
    processed_text = re.sub(r"(thank(s)*)", "", processed_text)
    return processed_text

### Read the data

In [10]:
def read_file(directory):
    all_files = os.listdir(directory)
    global text_dir
    for text_dir in all_files:
        if  text_dir.startswith("."):
            continue
        if os.path.isdir(text_dir):
            continue
        with open (directory + '/'  + text_dir, 'r', encoding='utf-8') as f: #change raw to the last folder
            text = f.read()
        print("Processing", text_dir)
        with open(directory + '/' + text_dir, 'w', encoding='utf-8') as p:
            p.write(tokenize(text))   
        
if __name__ == '__main__':
    directory = sys.argv[1]
    
read_file(filename)

Processing (CYH).txt
Processing (FE).txt
Processing (FMSA).txt
Processing (IHRT).txt
Processing (MFRM).txt


### Load the dataset

In [12]:
def load_data(data, label):
    for file in os.listdir(label):
        if not file.startswith('.'):
            file_path =  label + '/' +  file 
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
                data[text] = label
                
df = {}
load_data(df, filename)

Num = len(df)
print("Total doc:", Num)

Total doc: 5


### Exploratory sentiment analysis

In [13]:
#Analyze a Doc
corpus = textacy.Corpus(nlp, df)

stat = pd.DataFrame(columns = ['n_words', 'n_sents', 'n_long_words', 'ARI'])
doc_id = 0
for doc in corpus:
    ts = TextStats(doc)
    stat.loc[doc_id] = [ts.n_words, ts.n_sents, ts.n_long_words, ts.automated_readability_index]
    doc_id += 1
    
stat.describe()

,n_words,n_sents,n_long_words,ARI
count,5.000000,5.000000,5.000000,5.000000
mean,9205.000000,539.200000,1776.400000,7.550944
std,1863.216708,86.053472,238.365895,0.447144
min,8012.000000,475.000000,1526.000000,6.823956
25%,8113.000000,494.000000,1681.000000,7.483967
50%,8142.000000,509.000000,1686.000000,7.624507
75%,9373.000000,529.000000,1833.000000,7.880709
max,12385.000000,689.000000,2156.000000,7.941580


In [27]:
def sentiment_by_sentence(text, doc_id, default):
    sentiment_df = pd.DataFrame(columns = ["default", "doc_id", "polarity", "subjectivity", "progress"])
    tb = TextBlob(text)
    sent_id = 0
    num_sents = len(tb.sentences)
    for sent in tb.sentences:
        global sentiment
        sentiment = sent.sentiment
        sentiment_df.loc[sent_id] = [default, doc_id, sentiment.polarity, sentiment.subjectivity, round((sent_id+1)/num_sents,4)]
        sent_id += 1
    return sentiment_df

df_sentiment = pd.DataFrame(columns = ["default", "doc_id", "polarity", "subjectivity", "progress"])
doc_id = 1
for doc in df:
    df_sentiment = pd.concat([df_sentiment, sentiment_by_sentence(doc, doc_id, True)])
    doc_id += 1
    

df_sentiment.to_csv("SentimentData.csv") #saves it in the driectory

In [28]:
SentimentData = pd.read_csv("SentimentData.csv")
SentimentData.head(), SentimentData.tail()

(   Unnamed: 0  default  doc_id  polarity  subjectivity  progress
 0           0     True       1  0.125000      0.125000    0.0025
 1           1     True       1  0.178571      0.357143    0.0050
 2           2     True       1  0.000000      0.000000    0.0075
 3           3     True       1  0.000000      0.062500    0.0100
 4           4     True       1  0.000000      0.000000    0.0125,
       Unnamed: 0  default  doc_id  polarity  subjectivity  progress
 2101         475     True       5  0.250000      0.750000    0.9917
 2102         476     True       5  0.000000      0.000000    0.9938
 2103         477     True       5 -0.002083      0.852083    0.9958
 2104         478     True       5  0.004416      0.563052    0.9979
 2105         479     True       5  0.037857      0.469286    1.0000)